In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from transformers import AutoTokenizer
from transformers import GenerationConfig
import torch
import time
import evaluate
import numpy as np
import pandas as pd

In [2]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [5]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)



In [6]:
def print_number_of_trainable_modelparameters(model):

    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()

    return f"trainable model parameters: {trainable_model_params}\nall_model_params: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}% Model"

print(print_number_of_trainable_modelparameters(original_model))


trainable model parameters: 247577856
all_model_params: 247577856
percentage of trainable model parameters: 100.00% Model
